In [1]:
import requests # petición a al página
from bs4 import BeautifulSoup as bs # para extraer html
import random # para números aleatorios
import time # para hacer pausas mientras estoy scrapeando
import pandas as pd
import numpy as np
from selenium import webdriver # libreria que nos permite hacer web scrapping
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc # hacemos que sea imposible para la página web que detecte que somos una pc
import re

In [2]:
# Abrimos el explorador
browser = uc.Chrome()

In [108]:
# Buscamos el sitio del que queremos extraer los datos, entramos a la primera publicación (debemos primero extraer información de uno para que el programa luego automatice los siguientes)
# Copiamos link de la primera publicación
url = 'https://auto.mercadolibre.com.ar/MLA-1843928488-vw-amarok-dark-label-4x4-automatica-2015-ugarcia-_JM#position%3D1%26search_layout%3Dgrid%26type%3Ditem%26tracking_id%3Dfb14d815-6d30-4b00-bd17-cfe36b494f53'

In [109]:
# Le decimos al browser que vaya a esa url
browser.get(url)
time.sleep(5) # espera

In [208]:
# Podemos o darle a aceptar las cookies manualmente o codear para que me las acepte sola el browser (clik derecho, inpeccionar sobre el boton de aceptar cookies)
# Le doy a Ctrl + Shift + C para buscar elementos de la página manteniendo el cursor por encima de los mismos y haciendolé click
# Copiamos el id del botón y el nombre con el que se muestra el id

#browser.find_element("xpath",'//*[@data-testid="action:understood-button"]').click() # me acepta las cookies

In [110]:
# Una vez aceptada las cookies, podemos sacar el html de la página
html = browser.page_source

In [111]:
# Y ahora comenzamos a usar Beatiful Soup, que me acomoda el código html de la página para que sea más legible
soup = bs(html,'lxml') # 'lxml' es un tipo de espaciador que vamos a usar para simplificar el código

In [66]:
# Para cada cosa que quiera extraer de la página en concreto, lo que tengo que hacer es ir a inpeccionar, seleccionar el elemento que deseo extraer y luego copiar su tipo de clase primero y su contenido de clase después 

In [67]:
# Extraigo el Título
titulo = soup.find('h1',{'class':'ui-pdp-title'}).text # lo paso a texto
titulo

'Renault Clio 1.2 Mio Confort Plus Abc'

In [68]:
# Extraigo la Ubicación
ubicacion = soup.find('span',{'class':'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--SEMIBOLD'}).text.split('-')[-1:]
# pongo para que me separe los elementos según el guión, retornándome una lista de la cual puedo extraer los elementos que quiera (en este caso los 2 últimos por si agrega una locación muy específica al principio)
ubicacion

[' Córdoba']

In [69]:
# Extraigo el precio (variable a predecir)
precio = int(soup.find('span',{'class':'andes-money-amount__fraction'}).text.replace('.',''))
# reemplazo el punto por coma si busco un dato de tipo float o a vacío si busco uno de tipo int
if precio<400000: # apartado para tratar precios subidos en dólares
    precio = precio*1465 # tipo de cambio blue
precio

4600000

In [70]:
# Extraigo el año del modelo y cuántos km tiene (desp se extrae más facil)
#año_modelo_y_km_recorridos = soup.find('span',{'class':'ui-pdp-subtitle'}).text.split('·') # divido el año del modelo con los km recorridos con cuándo se publicó (intrascendente)
#año_modelo_y_km_recorridos
#primer_elemento = año_modelo_y_km_recorridos[0]  # me quedo con el primer elemento ('2023 | 0 km ')
#datos = primer_elemento.split('|') # divido el primer elemento en año y kilómetros recorridos
#año_modelo = int(datos[0].strip())  # convierto a int el año del modelo
#km_recorridos = int(datos[1].strip().split()[0].replace('.', '')) # convierto a int los km recorridos, eliminando el km
#print(año_modelo,km_recorridos)

In [112]:
# Si quiero extraer características de una caja de código (cuadro que engloba muchas características)
#c1 = soup.find('div',{'class':'ui-pdp-container__row ui-pdp-container__row--technical-specifications'}).text
c1 = soup.find('div',{'class':'ui-pdp-container__row ui-pdp-container__row--technical-specifications'}).text

# código del cuadro entero (para seleccionar elemento me sale una separación violeta) // código del lado del cuadro que me interesa
print(c1) # acá tengo la mitad que me interesa de cuadro

PrincipalesMarcaVolkswagenModeloAmarokAño2015Versión2.0 Cd Tdi 180cv 4x4 Dark Label AtColorAzulTipo de combustibleDiéselPuertas4TransmisiónAutomáticaMotor2.0Tipo de carroceríaPick-UpKilómetros104521 kmSeguridadFrenos ABSSíAirbag para conductor y pasajeroSíControl de estabilidadSíTracción ASRSíConfort y convenienciaPiloto automáticoSíComputadora de abordoSíPorta vasosSíInformación generalDirecciónHidráulicaRendimiento y dimensionesControl de tracción4x4Capacidad de personas5Potencia180 hpLargo x Altura x Ancho5254 mm x 1834 mm x 1944 mmDistancia entre ejes3095 mmCapacidad del tanque80 LVálvulas por cilindro4ExteriorLlantas de aleaciónSíPorta equipaje en techoSíInteriorTapizado de cueroSíVer todas las características


In [72]:
# Guardamos el tipo de transmisión
#indice_inicio = c1.find("Transmisión: ") + len("Transmisión: ")
#transmision = c1[indice_inicio:].strip()
#transmision

In [73]:
# Extraigo la otra mitad del cuadro
#c2 = soup.find('div',{'class':'ui-vpp-highlighted-specs__attribute-columns'}).find('div',{'ui-vpp-highlighted-specs__attribute-columns__column'}).text
#c2
patterns = {
    "Marca": r"Marca([A-Za-z\s]+)Modelo",
    "Modelo": r"Modelo([A-Za-z0-9\s]+)Año",
    "Año": r"Año(\d{4})",
    "Versión": r"Versión([A-Za-z0-9\s]+)Color",
    "Color": r"Color([A-Za-z\s]+)Tipo de combustible",
    "Tipo de combustible": r"Tipo de combustible([A-Za-z\s]+)Puertas",
    "Puertas": r"Puertas(\d+)",
    "Transmisión": r"Transmisión([A-Za-z\s]+)Motor",
    "Motor": r"Motor([0-9.]+)Tipo de carrocería",
    "Tipo de carrocería": r"Tipo de carrocería([A-Za-z\s]+)Kilómetros",
    "Kilómetros": r"Kilómetros([\d]+) km",
    "Único dueño": r"Único dueño([A-Za-z\s]+)Confort y conveniencia",
    "Piloto automático": r"Piloto automático([A-Za-z\s]+)Aire acondicionado",
    "Aire acondicionado": r"Aire acondicionado([A-Za-z\s]+)Alarma de luces encendidas",
    "Dirección": r"Dirección([A-Za-z\s]+)Rendimiento y dimensiones",
    "Control de tracción": r"Control de tracción([A-Za-z\s]+)Capacidad de personas",
    "Capacidad de personas": r"Capacidad de personas(\d+)",
    "Potencia": r"Potencia(\d+)"
}


In [123]:
# (Actualizar hasta que te lo tome)
# Guardamos las características del auto 
import re

# Texto de entrada
texto = c1
# Diccionario para almacenar las variables y su contenido
variables = {}

# Definir las expresiones regulares para cada variable
patterns = {
    "Marca": r"Marca([A-Za-z\s]+)Modelo",
    "Modelo": r"Modelo([A-Za-z0-9\s]+)Año",
    "Año": r"Año(\d{4})",
    "Versión": r"Versión([\w\s.]+)Color",
    "Color": r"Color([A-Za-z\s]+)Tipo de combustible",
    "Tipo de combustible": r"Tipo de combustible([A-Za-z\séÉíÍúÚóÓáÁñÑ]+)Puertas",
    "Puertas": r"Puertas(\d+)",
    "Transmisión": r"Transmisi[óo]n([A-Za-z\séÉíÍúÚóÓáÁñÑ]+)Motor",
    "Motor": r"Motor([\d.]+)Tipo de carrocería",
    "Tipo de carrocería": r"Tipo de carrocería([A-Za-z\s-]+)Kilómetros",
    "Kilómetros": r"Kilómetros([\d]+) km",
    "Dirección": r"Dirección([A-Za-z\séÉíÍúÚóÓáÁñÑ]+)Rendimiento y dimensiones",
    "Control de tracción": r"Control de tracción([\w\s]+)Capacidad de personas",
    "Capacidad de personas": r"Capacidad de personas(\d+)",
    "Potencia": r"Potencia(\d+) hp"
}

# Extraer y almacenar los valores en el diccionario
for key, pattern in patterns.items():
    match = re.search(pattern, texto)
    if match:
        variables[key] = match.group(1).strip()

# Imprimir los resultados
for key, value in variables.items():
    print(f"{key}: {value}")

Marca: Volkswagen
Modelo: Amarok
Año: 2015
Versión: 2.0 Cd Tdi 180cv 4x4 Dark Label At
Color: Azul
Tipo de combustible: Diésel
Puertas: 4
Transmisión: Automática
Motor: 2.0
Tipo de carrocería: Pick-Up
Kilómetros: 104521
Dirección: Hidráulica
Control de tracción: 4x4
Capacidad de personas: 5
Potencia: 180


In [75]:
# Extraigo la característica específica de mi mitad de cuadro
# caract_basicas = [caract.text.strip() for caract in c1.find_all('li')] # 'cambiamos li por el <separador que tengamos>'
# caract_básicas

In [18]:
# Automaticemos para todos los autos
url_grande = 'https://autos.mercadolibre.com.ar/autos_ITEM*CONDITION_2230581_NoIndex_True#applied_filter_id%3DITEM_CONDITION%26applied_filter_name%3DCondici%C3%B3n%26applied_filter_order%3D11%26applied_value_id%3D2230581%26applied_value_name%3DUsado%26applied_value_order%3D2%26applied_value_results%3D78280%26is_custom%3Dfalse'

In [19]:
browser.get(url_grande)
time.sleep(10) # espera

In [20]:
html = browser.page_source

In [21]:
soup = bs(html,'lxml')
soup

<html lang="es-AR"><head><link href="https://www.google-analytics.com" rel="preconnect"/><link href="https://www.google.com" rel="preconnect"/><link href="https://data.mercadolibre.com" rel="preconnect"/><link href="https://http2.mlstatic.com" rel="preconnect"/><link href="https://stats.g.doubleclick.net" rel="preconnect"/><link href="https://analytics.mercadolibre.com.ar" rel="preconnect"/><link href="https://analytics.mercadolibre.com" rel="preconnect"/><link href="https://www.google.com.ar" rel="preconnect"/></head><body><iframe src='https://http2.mlstatic.com/storage/tag-manager/MLA.html?timehash=28681500&amp;platform=STD#{"items":"BILLBOARD_INTERVENTION,POLYCARD,POLYCARD,POLYCARD,POLYCARD","categoryId":[","],"query":"autos","business":"CLASSIFIED","pageVertical":"MOTORS"}' style="width: 0px; height: 0px; border: 0px; position: absolute;"></iframe><script async="" src="https://http2.mlstatic.com/frontend-assets/perfill-agent/3.12.3/perfill-agent.min.js"></script><script nonce="" ty

In [25]:
# Identifico articulos y lista, y a partir de cada artículo en la lista voy a ir iterando
# Busco algo que tengan en común todos los artículos a la venta (li class) y todos estan dentro de una clase más grande (section)
# Ponemos que busque dentro de la lista de articulos cada uno por separado 
# articulos = soup.find('section',{'class':'ui-search-results'}).find_all('section')
# articulos = soup.find('section',{'class':'ui-search-results'}).find_all('div') # anda
#articulos = soup.find('section',{'class':'ui-search-results'}).find_all('form') # pagina general
articulos = soup.find('section',{'class':'ui-search-results'}).find_all('a', href=True)
articulos

[<a class="ui-search-billboard__logo-image-link" href="https://auto.mercadolibre.com.ar/MLA-1426653539-bmw-x3-xdrive30i-2021-25000-km-natalio-automotores-_JM#position%3D-1%26search_layout%3Dgrid%26type%3Ditem%26tracking_id%3D2d5fc57b-c690-4d63-b4c6-0b31a5de050cBB=1&amp;c_component=bb&amp;c_pos=1&amp;c_campaign=bb&amp;c_client=search" tabindex="-1"><figure class="ui-search-billboard__logo-image-container ui-search-billboard__logo-image-container--billboard_intervention"><img alt="Bmw X3 Xdrive30i 2021 - 25.000 Km - Natalio Automotores" class="ui-search-billboard__logo-image ui-search-billboard__logo-image--billboard_intervention" decoding="sync" fetchpriority="high" src="https://http2.mlstatic.com/storage/vis-accounts/232631093_vip-ca1589fd-5e1c-40f5-8b37-6f146216de07.jpg"/></figure></a>,
 <a class="ui-search-billboard__link" href="https://auto.mercadolibre.com.ar/MLA-1426653539-bmw-x3-xdrive30i-2021-25000-km-natalio-automotores-_JM#position%3D-1%26search_layout%3Dgrid%26type%3Ditem%26t

In [27]:
import re
# Identifico el id de cada artículo
#id_autos = [articulo.get('href') for articulo in articulos if 'MLA' in articulo]
id_autos = [articulo['href'] for articulo in articulos if re.search(r'MLA', articulo['href'])]
len(id_autos)

86

In [32]:
# Eliminamos los Nulos
id_autos = [autos for autos in id_autos if autos is not None]
#id_autos = [elemento for elemento in id_autos if elemento.endswith(':')]
id_autos = [item.split('/')[-1] for item in id_autos]
def extraer_mla(cadena):
    match = re.search(r'MLA-\d+', cadena)
    if match:
        return match.group(0)
    return None
id_autos = [extraer_mla(cadena) for cadena in id_autos]
id_autos = list(set(id_autos))
len(id_autos)
id_autos

['MLA-1426653539',
 'MLA-1426653539',
 'MLA-1426653539',
 'MLA-1434915291',
 'MLA-1434915291',
 'MLA-1434915291',
 'MLA-1836654478',
 'MLA-1433982419',
 'MLA-1433982419',
 'MLA-1433982419',
 'MLA-1433982419',
 'MLA-1433982419',
 'MLA-1433982419',
 'MLA-1433982419',
 'MLA-1433982419',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844747594',
 'MLA-1844734556',
 'MLA-1844747460',
 'MLA-1844429776',
 'MLA-1844773176',
 'MLA-1844760202',
 'MLA-1437547375',
 'MLA-1844759768',
 'MLA-1437545885',
 'MLA-1437493647',
 'MLA-1844427022',
 'MLA-1437519389',
 'MLA-184442

In [36]:
id_autos = list(set(id_autos))
len(id_autos)

50

In [30]:
#pagina_actual = soup.find('div',{'class':'ui-search'}).text
pagina_actual = int(soup.find('div',{'class':'ui-search'}).find('ul',{'class':'andes-pagination ui-search-andes-pagination andes-pagination--large'}).find('li',{'class':'andes-pagination__button--current'}).text)
print(pagina_actual) 

1


In [369]:
# Ahora automatizamos la extracción para todos los datos de la página
# Inicia el navegador
#browser = uc.Chrome()
busqueda = 'autos'
modelo = 'strada' # BUSCAR POR MODELOS
rango_precios = 'PriceRange_58000USD-0USD'# PARA CUANDO RECOPILEMOS POR PRECIO
# busqueda = busqueda.replace(' ','-') poner con guion
x=1
ids = []
cantidad_autos = 0
def extraer_mla(cadena):
        match = re.search(r'MLA-\d+', cadena)
        if match:
            return match.group(0)
        return None

while cantidad_autos<=300: # Poner un número menor al de los resultados del filtro de búsqueda (no mayor a 2016, cant máxima de publicaciones por criterio)
    if cantidad_autos == 0:
        #url = f'https://{busqueda}.mercadolibre.com.ar/{busqueda}_{rango_precios}_ITEM*CONDITION_2230581_NoIndex_True' # POR RANGO DE PRECIOS
        url = f'https://{busqueda}.mercadolibre.com.ar/{modelo}/{busqueda}_ITEM*CONDITION_2230581_NoIndex_True' # POR MODELO
    else:
        #url = f'https://{busqueda}.mercadolibre.com.ar/{busqueda}_Desde_{cantidad_autos}_{rango_precios}_ITEM*CONDITION_2230581_NoIndex_True' # POR RANGO DE PRECIOS
        url = f'https://{busqueda}.mercadolibre.com.ar/{modelo}/{busqueda}_Desde_{cantidad_autos}_ITEM*CONDITION_2230581_NoIndex_True' # POR MODELO

    browser.get(url)
    time.sleep(random.randint(10, 12))
    
    try:
            browser.find_element("xpath",'//*[@data-testid="action:understood-button"]').click() # me acepta las cookies
    except:
        pass

    html = browser.page_source
    soup = bs(html, 'lxml')
    #pagina_actual = int(soup.find('div',{'class':'ui-search'}).find('ul',{'class':'andes-pagination ui-search-andes-pagination andes-pagination--large'}).find('li',{'class':'andes-pagination__button--current'}).text)
    # el num de pag esta dentro de un li que esta dentro de la misma de la que sacamos los articulos, y luego el li seleccionado tiene un '.andes-pagination__button--current'
    #if x == pagina_actual:
       #articulos = soup.find('section',{'class':'ui-search-results'}).find_all('form')
    articulos = soup.find('section',{'class':'ui-search-results'}).find_all('a', href=True) 
    #else:
    #    break # seria break pero en ML se pueden particionar las paginas de busqueda por cant autos menores

    x = x+1
    nuevos_ids = []
    for articulo in articulos:
        href = articulo['href']
        if 'MLA' in href:
            id_auto = extraer_mla(href)
            if id_auto:
                nuevos_ids.append(id_auto)
                print(id_auto)
            time.sleep(random.randint(1, 3))

    # Eliminar duplicados y mantener solo IDs únicos
    nuevos_ids = list(set(nuevos_ids)) # eliminar dupl de la nueva pag
    ids.extend(nuevos_ids) # agg a ids orig
    ids = list(set(ids)) #  eliminar dupl totales
    print(len(ids))
    cantidad_autos = len(ids)

    #for articulo in articulos:
        # id_autos = articulo.get('action')
        #id_autos = [articulo['href'] for articulo in articulos if re.search(r'MLA', articulo['href'])]
        #ids.append(id_autos)
        #print(id_autos)
        #time.sleep(random.randint(1,3))
    
   # ids = [autos for autos in ids if autos is not None]
   # ids = [item.split('/')[-1] for item in ids]
    
   # ids = [extraer_mla(cadena) for cadena in ids]
   # ids = list(set(ids))
   # print(len(ids))
   # cantidad_autos = len(ids)
    
# Extraer de a una especifidad por vez, sabiendo que el número máximo de datos a extraer por búsqueda es de 2018 (por modelo es excelente idea)

MLA-1438420479
MLA-1438183521
MLA-1438042741
MLA-1839144520
MLA-1839143662
MLA-1434235345
MLA-1431632705
MLA-1792692920
MLA-1423936355
MLA-1718212426
MLA-1434843587
MLA-1438393075
MLA-1848665962
MLA-1842007282
MLA-1435417443
MLA-1436767723
MLA-1826810342
MLA-1434968021
MLA-1438214345
MLA-1843971924
MLA-1435251619
MLA-1826763098
MLA-1435045031
MLA-1849325480
MLA-1437196743
MLA-1435837937
MLA-1433195169
MLA-1433272045
MLA-1847754106
MLA-1770384218
MLA-1438261993
MLA-1847848234
MLA-1846932246
MLA-1437604925
MLA-1438321265
MLA-1435957511
MLA-1842272146
MLA-1436513343
MLA-1844751564
MLA-1825663788
MLA-1828512998
MLA-1434943797
MLA-1852354490
MLA-1435684951
MLA-1826425694
MLA-1429341227
MLA-1430081879
MLA-1427521381
48
MLA-1427521381
MLA-1847057506
MLA-1847081976
MLA-1847006142
MLA-1801710100
MLA-1837905872
MLA-1434547795
MLA-1427450219
MLA-1760031960
MLA-1850648760
MLA-1428167113
MLA-1733512262
MLA-1438308509
MLA-1785189936
MLA-1734271158
MLA-1841564804
MLA-1435868949
MLA-1432561211
MLA-182

In [370]:
ids_autos = pd.DataFrame(ids) # cambiarle el nombre y poner el modelo del auto (ids_modelo) #(ids)
ids_autos.columns = ['id']
ids_autos

,id
0,MLA-1787234818
1,MLA-1834304896
2,MLA-1422175491
3,MLA-1438300297
4,MLA-1828522820
...,...
310,MLA-1432108193
311,MLA-1428545271
312,MLA-1829684754
313,MLA-1438261993


In [371]:
# CAMBIAR EL MODELO
ids_autos.to_csv('strada_ids_autos.csv',index = False) # guardarlo también como un csv de ese modelo (despues unir todos los modelos) CAMBIAR SEGUN MODELO

In [372]:
autos = pd.Series() # creamos la serie para guardar cada auto con sus características (vacío), RESETEAR CON CADA MODELO

In [373]:
df_autos = pd.DataFrame() # USAR 
def parsear_autos(id_auto):
    try:
        print('\n Auto id: ' + id_auto)
        url = f'https://auto.mercadolibre.com.ar/{id_auto}'
        browser.get(url)
        html = browser.page_source
        soup = bs(html, 'lxml')
        
        titulo = soup.find('h1', {'class': 'ui-pdp-title'})
        if titulo:
            titulo = titulo.text.strip()
            print('\n Título: ' + titulo)
        else:
            print('Título no encontrado')
            return None
        
        ubicacion = soup.find('span', {'class': 'ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--SEMIBOLD'})
        if ubicacion:
            ubicacion_text = ubicacion.text.strip()
            if '-' in ubicacion_text:
                ubicacion = ubicacion_text.split('-')[-1].strip()
            else:
                ubicacion = ubicacion_text
                print('\n Ubicación: ' + ubicacion)
        else:
            ubicacion = 'Ubicación no encontrada'
            print(ubicacion)
        
        # Encuentra el símbolo de moneda y precio
        currency_symbol = soup.find('span', {'class': 'andes-money-amount__currency-symbol'})
        precio_text = soup.find('span', {'class': 'andes-money-amount__fraction'})
        if currency_symbol and precio_text:
            precio_text = precio_text.text.replace('.', '')
            if currency_symbol.text.strip() == 'US$':
                precio = int(precio_text) * 1500
            else:
                precio = int(precio_text)
            print('\n Precio: ' + str(precio))
        else:
            print('Precio no encontrado')
            return None
        
        # Extracción de características del auto
        c1 = soup.find('div', {'class': 'ui-pdp-container__row ui-pdp-container__row--technical-specifications'})
        if c1:
            c1 = c1.text
            # Aquí deberías implementar la lógica para extraer las características según los patrones
            variables = {}
            patterns = {
            "Marca": r"Marca([A-Za-z\s]+)Modelo",
            "Modelo": r"Modelo([A-Za-z0-9\séÉíÍúÚóÓáÁñÑ.-]+)Año",
            "Año": r"Año(\d{4})",
            "Versión": r"Versión([\w0-9\séÉíÍúÚóÓáÁñÑ.-]+?)(Color|Tipo de combustible)",
            "Color": r"Color([A-Za-z\s]+)Tipo de combustible",
            "Tipo de combustible": r"Tipo de combustible([A-Za-z/\séÉíÍúÚóÓáÁñÑ]+)Puertas",
            "Puertas": r"Puertas(\d+)",
            "Transmisión": r"Transmisi[óo]n([A-Za-z\séÉíÍúÚóÓáÁñÑ]+)Motor",
            "Motor": r"Motor([\d.]+?)(Tipo de carrocería|Kilómetros)",
            "Tipo de carrocería": r"Tipo de carrocería([A-Za-z\séÉíÍúÚóÓáÁñÑ-]+?)(Kilómetros|Con cámara de retroceso)", 
            "Kilómetros": r"Kilómetros([\d]+) km",
            "Dirección": r"Dirección([A-Za-z\séÉíÍúÚóÓáÁñÑ]+?)(Rendimiento y dimensiones|Alarma)", # si no aparece dirección seguro es Hidráulica
            "Control de tracción": r"Control de tracción([\w\s]+?)(Capacidad de personas|Tipo de batería)", # VER "Con modo Eco"
            "Capacidad de personas": r"Capacidad de personas(\d+)",
            "Potencia": r"Potencia(\d+) hp"
            }

            for key, pattern in patterns.items():
                match = re.search(pattern, c1)
                if match:
                    variables[key] = match.group(1).strip()
                    print(f'{key}: {variables[key]}')
                else:
                    print(f'{key} no encontrado')
                    variables[key] = None
            
            # Crear la serie autos con los datos
            autos = pd.Series({
                'Título': titulo,
                'Ubicación': ubicacion,
                'Marca': variables.get('Marca', None),
                'Modelo': variables.get('Modelo', None),
                'Año': variables.get('Año', None),
                'Versión': variables.get('Versión', None),
                'Color': variables.get('Color', None),
                'Tipo de combustible': variables.get('Tipo de combustible', None),
                'Puertas': variables.get('Puertas', None),
                'Transmisión': variables.get('Transmisión', None),
                'Motor': variables.get('Motor', None),
                'Tipo de carrocería': variables.get('Tipo de carrocería', None),
                'Kilómetros': variables.get('Kilómetros', None),
                'Único dueño': variables.get('Único dueño', None),
                'Piloto automático': variables.get('Piloto automático', None),
                'Aire acondicionado': variables.get('Aire acondicionado', None),
                'Dirección': variables.get('Dirección', None),
                'Control de tracción': variables.get('Control de tracción', None),
                'Capacidad de personas': variables.get('Capacidad de personas', None),
                'Potencia': variables.get('Potencia', None),
                'Precio': precio
            })
            
            df_autos = pd.DataFrame(autos.T) 
            return (df_autos.T) 
            
        else:
            print('Características del auto no encontradas')
            return None

    except Exception as e:
        print(f"Error al procesar el auto {id_auto}: {str(e)}")
        return None

In [268]:
# DESPUES CON CADA CSV CONSTRUIR LOS DISTINTOS DF Y UNIRLOS TODOS EN UNO AMPLIO

In [374]:
df_autos = pd.DataFrame() # por si hay que resetear la ejecucion
df_autos = parsear_autos(ids_autos.iloc[0].id) 

for i in range(1,len(ids)): 
    df_autos = pd.concat([df_autos,parsear_autos(ids[i])]) 
    time.sleep(random.randint(4,8))


 Auto id: MLA-1787234818

 Título: Fiat Strada 1.6 Adventure Cd Pack Top
Ubicación no encontrada

 Precio: 15000000
Marca: Fiat
Modelo: Strada
Año: 2016
Versión: 1.6 Adventure Cd Pack Top
Color no encontrado
Tipo de combustible: Nafta
Puertas: 3
Transmisión: Manual
Motor: 1.6
Tipo de carrocería: Pick-Up
Kilómetros: 115000
Dirección: Hidráulica
Control de tracción: Delantera
Capacidad de personas: 4
Potencia: 115

 Auto id: MLA-1834304896

 Título: Fiat Strada 1.6 Adventure Cd Pack Top
Ubicación no encontrada

 Precio: 15000
Marca: Fiat
Modelo: Strada
Año: 2020
Versión: 1.6 Adventure Cd Pack Top
Color: Gris
Tipo de combustible: Nafta
Puertas: 3
Transmisión: Manual
Motor: 1.6
Tipo de carrocería: Pick-Up
Kilómetros: 72000
Dirección: Hidráulica
Control de tracción: Delantera
Capacidad de personas: 4
Potencia: 115

 Auto id: MLA-1422175491

 Título: Fiat Strada 1.4 Working Cd C/aa + Pack Seg
Ubicación no encontrada

 Precio: 10000
Marca: Fiat
Modelo: Strada
Año: 2013
Versión no encontrado


In [375]:
df_autos

,Título,Ubicación,Marca,Modelo,Año,Versión,Color,Tipo de combustible,Puertas,Transmisión,...,Tipo de carrocería,Kilómetros,Único dueño,Piloto automático,Aire acondicionado,Dirección,Control de tracción,Capacidad de personas,Potencia,Precio
0,Fiat Strada 1.6 Adventure Cd Pack Top,Ubicación no encontrada,Fiat,Strada,2016,1.6 Adventure Cd Pack Top,None,Nafta,3,Manual,...,Pick-Up,115000,None,None,None,Hidráulica,Delantera,4,115,15000000
0,Fiat Strada 1.6 Adventure Cd Pack Top,Ubicación no encontrada,Fiat,Strada,2020,1.6 Adventure Cd Pack Top,Gris,Nafta,3,Manual,...,Pick-Up,72000,None,None,None,Hidráulica,Delantera,4,115,15000
0,Fiat Strada 1.4 Working Cd C/aa + Pack Seg,Ubicación no encontrada,Fiat,Strada,2013,None,None,Nafta,2,Manual,...,Pick-Up,91500,None,None,None,Hidráulica,Delantera,5,87,10000
0,Fiat Strada 1.4 Working Cs,Ubicación no encontrada,Fiat,Strada,2015,1.4 Working Cs,Blanco,Nafta,2,Manual,...,Pick-Up,118000,None,None,None,Hidráulica,Delantera,2,87,11900000
0,Fiat Strada 1.4 Trekking,Ubicación no encontrada,Fiat,Strada,2012,1.4 Trekking,Verde,Nafta,2,Manual,...,Pick-Up,150000,None,None,None,Hidráulica,Delantera,5,87,8500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Fiat Strada 1.6 Adventure Cd,Ubicación no encontrada,Fiat,Strada,2016,1.6 Adventure Cd,Negro,Nafta,3,None,...,None,44134,None,None,None,None,None,None,None,18000000
0,Fiat Strada 1.3 Volcano,Ubicación no encontrada,Fiat,Strada,2022,1.3 Volcano,Blanco,Nafta,3,Manual,...,Pick-Up,28000,None,None,None,Hidráulica,Delantera,5,99,17500
0,Fiat Strada 1.3 Mjet Trekking Cs + Aa,Bs.As. G.B.A. Oeste,Fiat,Strada,2011,None,Gris,Diésel,2,Manual,...,Pick-Up,170000,None,None,None,Hidráulica,Delantera,5,87,6000
0,Fiat Strada 1.4 8v Freedom Cd,Ubicación no encontrada,Fiat,Strada,2021,1.4 8v Freedom Cd,Gris,Nafta,3,Manual,...,Pick-Up,49000,None,None,None,Hidráulica,Delantera,5,85,18490000


In [376]:
# Reseteamos el index
df_autos.reset_index(drop=True, inplace=True)
df_autos

,Título,Ubicación,Marca,Modelo,Año,Versión,Color,Tipo de combustible,Puertas,Transmisión,...,Tipo de carrocería,Kilómetros,Único dueño,Piloto automático,Aire acondicionado,Dirección,Control de tracción,Capacidad de personas,Potencia,Precio
0,Fiat Strada 1.6 Adventure Cd Pack Top,Ubicación no encontrada,Fiat,Strada,2016,1.6 Adventure Cd Pack Top,None,Nafta,3,Manual,...,Pick-Up,115000,None,None,None,Hidráulica,Delantera,4,115,15000000
1,Fiat Strada 1.6 Adventure Cd Pack Top,Ubicación no encontrada,Fiat,Strada,2020,1.6 Adventure Cd Pack Top,Gris,Nafta,3,Manual,...,Pick-Up,72000,None,None,None,Hidráulica,Delantera,4,115,15000
2,Fiat Strada 1.4 Working Cd C/aa + Pack Seg,Ubicación no encontrada,Fiat,Strada,2013,None,None,Nafta,2,Manual,...,Pick-Up,91500,None,None,None,Hidráulica,Delantera,5,87,10000
3,Fiat Strada 1.4 Working Cs,Ubicación no encontrada,Fiat,Strada,2015,1.4 Working Cs,Blanco,Nafta,2,Manual,...,Pick-Up,118000,None,None,None,Hidráulica,Delantera,2,87,11900000
4,Fiat Strada 1.4 Trekking,Ubicación no encontrada,Fiat,Strada,2012,1.4 Trekking,Verde,Nafta,2,Manual,...,Pick-Up,150000,None,None,None,Hidráulica,Delantera,5,87,8500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,Fiat Strada 1.6 Adventure Cd,Ubicación no encontrada,Fiat,Strada,2016,1.6 Adventure Cd,Negro,Nafta,3,None,...,None,44134,None,None,None,None,None,None,None,18000000
311,Fiat Strada 1.3 Volcano,Ubicación no encontrada,Fiat,Strada,2022,1.3 Volcano,Blanco,Nafta,3,Manual,...,Pick-Up,28000,None,None,None,Hidráulica,Delantera,5,99,17500
312,Fiat Strada 1.3 Mjet Trekking Cs + Aa,Bs.As. G.B.A. Oeste,Fiat,Strada,2011,None,Gris,Diésel,2,Manual,...,Pick-Up,170000,None,None,None,Hidráulica,Delantera,5,87,6000
313,Fiat Strada 1.4 8v Freedom Cd,Ubicación no encontrada,Fiat,Strada,2021,1.4 8v Freedom Cd,Gris,Nafta,3,Manual,...,Pick-Up,49000,None,None,None,Hidráulica,Delantera,5,85,18490000


In [377]:
# Guardamos en csv el df
# CAMBIAR EL MODELO
df_autos.to_csv('autos_strada_csv', index= False, sep= ';', encoding='utf-16')